# Bibtex to DF and tidy

* df
* author list
* publisher list
* journal list
* files
* normalization

item 241 has weird annot

## Provenance
* Created new 2025-05-16 at Bignor!

In [35]:
from pathlib import Path 
import pandas as pd
import re
import scratch as sc
import great2_bibtex as scbt
from functools import partial
from greater_tables import GT
fGT = partial(GT, large_ok=True)
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 0 ns (started: 2025-05-18 17:19:14 +01:00)


## Manual read

In [106]:
p = Path('books.bib')
p = Path('library.bib')
p.exists()

True

time: 0 ns (started: 2025-05-18 17:44:45 +01:00)


In [265]:
b = sc.Bib2df(p, True)
len(b.df)

6118

time: 313 ms (started: 2025-05-18 21:24:07 +01:00)


In [262]:
b.contents()

,column,nonna,distinct
0,type,6118,10
1,tag,6118,6076
2,abstract,2518,2376
3,author,6113,4856
4,doi,2687,2538
5,file,5951,5917
6,issn,2425,826
7,journal,3475,1224
8,keywords,2999,2052
9,pages,3885,2748


time: 235 ms (started: 2025-05-18 21:20:25 +01:00)


In [ ]:
# run fillna = False
b.contents(True)

6118

time: 15 ms (started: 2025-05-18 21:18:34 +01:00)


In [266]:
b.df.query('tag == "Doyle2012"').T

,671
type,article
tag,Doyle2012
abstract,
author,"Doyle, John Q and Presentation, Conference Call"
doi,
file,":C\:/S/Library/Doyle, Presentation/2012_Americ..."
issn,
journal,
keywords,
pages,


time: 31 ms (started: 2025-05-18 21:24:13 +01:00)


In [267]:
s = b.txt.find('article{Mildenhall2006')
s

187989

time: 0 ns (started: 2025-05-18 21:27:47 +01:00)


In [268]:
t = b.stxt[241]
t

'article{Mildenhall2006,\nannote = {From Duplicate 2 (\n\nActuarial Geometry\n\n- Mildenhall, Stephen J )\n\n\n\n\nFrom Duplicate 1 (\n\n\nActuarial Geometry\n\n\n- Mildenhall, Stephen J )\n\n\n\n\nFrom Duplicate 2 (\n\n\nActuarial Geometry\n\n\n- Mildenhall, Stephen J )\n\n},\nauthor = {Mildenhall, Stephen J},\nfile = {:C\\:/S/Library/Mildenhall/2006_Actuarial Geometry (working).pdf:pdf},\nkeywords = {capital allocation,capital determination,game,risk measure},\nnumber = {1254},\npages = {312--381},\ntitle = {{Actuarial Geometry (working)}},\nyear = {2006}\n}\n'

time: 15 ms (started: 2025-05-18 21:27:53 +01:00)


In [270]:
b.df.loc[241].T

type                                                       article
tag                                                 Mildenhall2006
abstract                                                          
author                                       Mildenhall, Stephen J
doi                                                               
file             :C\:/S/Library/Mildenhall/2006_Actuarial Geome...
issn                                                              
journal                                                           
keywords         capital allocation,capital determination,game,...
pages                                                     312--381
title                               {Actuarial Geometry (working)}
volume                                                            
year                                                          2006
isbn                                                              
publisher                                                     

time: 0 ns (started: 2025-05-18 21:28:20 +01:00)


## File demographics

In [272]:
b.contents()

,column,nonna,distinct
0,type,6118,10
1,tag,6118,6076
2,abstract,6118,2377
3,author,6118,4857
4,doi,6118,2539
5,file,6118,5918
6,issn,6118,827
7,journal,6118,1225
8,keywords,6118,2053
9,pages,6118,2749


time: 172 ms (started: 2025-05-18 21:29:19 +01:00)


In [ ]:
b.contents(True)

### Searching

In [273]:
b.df.head()

,type,tag,abstract,author,doi,file,issn,journal,keywords,pages,...,month,annote,primaryClass,address,editor,series,chapter,school,organization,howpublished
1,article,Schmidt2003,It is shown that the arbitrage free portfolio ...,"Schmidt, Anatoly B.",10.1016/S0378-4371(02)01551-0,:C\:/S/Library/Schmidt/2003_The true invariant...,03784371,Physica A: Statistical Mechanics and its Appli...,"Arbitrage free portfolio,Option pricing",535--538,...,,,,,,,,,,
2,book,Bahnemann2015,,"Bahnemann, David",,:C\:/S/Library/Bahnemann/2015_Distributions fo...,,,,,...,,,,,,,,,,
3,techreport,Khoshnevisan2011,,"Khoshnevisan, Davar",,:C\:/S/Library/Khoshnevisan/2011_Topics in Pro...,,,,,...,,,,,,,,,,
4,article,Mahul2003,This paper examines optimal catastrophic risk ...,"Mahul, Olivier",,:C\:/S/Library/Mahul/2003_Efficient Risk Shari...,,Paper presented at the 2003 NBER Insurance Pro...,"cataclysm,limited liability,partial insurance,...",,...,,,,,,,,,,
5,article,Jiang2025,,"Jiang, By John X and Koo, David S and Wang, Is...",,":C\:/S/Library/Jiang, Koo, Wang/2025_The Road ...",,The International Journal of Accounting,PMIR,,...,,,,,,,,,,


time: 15 ms (started: 2025-05-18 21:29:39 +01:00)


In [274]:
b.df.loc[b.df.author.str.contains('Bloch|Deligne', regex=True, case=False).fillna(False)]

,type,tag,abstract,author,doi,file,issn,journal,keywords,pages,...,month,annote,primaryClass,address,editor,series,chapter,school,organization,howpublished
754,article,Alarcon2016,It is nearly impossible to separate two interl...,"Alarc{\'{o}}n, H{\'{e}}ctor and Salez, Thomas ...",10.1103/PhysRevLett.116.015502,:C\:/S/Library/Alarc{\'{o}}n et al/2016_Self-A...,10797114,Physical Review Letters,,1--5,...,,,,,,,,,,
1381,article,Bloch2014,,"Bloch, Spencer",,:C\:/S/Library/Bloch/2014_Algebraic Cycles.pdf...,,,,,...,,,,,,,,,,
3086,article,Deligne2012,,"Deligne, Pierre and Sullivan, Pierre",,":C\:/S/Library/Deligne, Sullivan/2012_Division...",,,,,...,,,,,,,,,,
3837,article,Delignette-Muller2015,The package fitdistrplus provides functions fo...,"Delignette-Muller, Marie Laure and Dutang, Chr...",10.18637/jss.v064.i04,":C\:/S/Library/Delignette-Muller, Dutang/2015_...",1548-7660,Journal of Statistical Software,"bootstrap,censored data,distributions,maximum ...",,...,,,,,,,,,,
5406,article,Bloch1984,,"Bloch, Spencer",10.1515/crll.1984.350.94,:C\:/S/Library/Bloch/1984_Algebraic Cycles and...,14355345,Journal fur die Reine und Angewandte Mathematik,,94--108,...,,,,,,,,,,


time: 47 ms (started: 2025-05-18 21:29:46 +01:00)


### Duplicated tag

In [280]:
fGT(b.df.loc[b.df.tag.duplicated(keep=False), ['tag', 'author', 'title']].sort_values('tag'), tabs=[5, 15, 25, 40])

index,tag,author,title
"2,288",Aliprantis2006,"Aliprantis, Charalambos D and Border, Kim",{Infinite Dimensional Analysis: A Hitchhiker's Guide}
742,Aliprantis2006,"Aliprantis, Charalambos D and Burkinshaw, Owen",{Positive Operators}
418,AonBenfield2016,"Mildenhall, Stephen J",{Insurance Risk Study: The First Ten Years}
"5,541",AonBenfield2016,{Aon Benfield},{Cyber Opportunity and Future Growth}
"1,793",Avanzi2011,"Avanzi, Benjamin",{Optimal dividends and capital injections in the dual model with diffusions}
"3,225",Avanzi2011,"Avanzi, Benjamin and Cassar, Luke C and Wong, Bernard",{Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy Copulas Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy Copulas}
"2,938",Babbel1997,"Babbel, David F",{Review of Two Paradigms for the Market Value of Liabilities by Reitano}
"2,023",Babbel1997,"Babbel, David F.","{Discussion of ``Two Paradigms for The Market Value of Liabilities, Robert R. Reitano''}"
"3,857",Bailey1967,"Bailey, Robert A",{Underwriting Profit From Investments}
"4,082",Bailey1967,"Bailey, Robert A",{Underwriting Profits From Investments}


time: 109 ms (started: 2025-05-18 21:31:24 +01:00)


### Distinct Journals

In [316]:
j = b.distinct('journal')
j[:10]

['116 Mich. L. Re.',
 '1Mathematical Intelligencer',
 '1Stochastic Processes and their Applications',
 '2003 IEEE Computer Society Conference on Computer Vision and Pattern Recognition, 2003. Proceedings.',
 '2012 International Conference on Systems and Informatics (ICSAI2012)',
 '2Variance',
 '30th Annual Symposium on Foundations of Computer Science',
 '31st International Congress of Actuaries',
 '34th International Conference on Machine Learning, ICML 2017',
 'A Modern Course on Statistical Distributions in Scientific Work']

time: 16 ms (started: 2025-05-18 21:51:12 +01:00)


In [320]:
fGT(b.df.journal.value_counts().head(100))

journal,count
,"2,643"
Insurance: Mathematics and Economics,145
Journal of Risk and Insurance,136
ASTIN Bulletin,108
SSRN Electronic Journal,60
North American Actuarial Journal,58
Mathematical Finance,50
Risk Management and Insurance Review,46
Journal of Finance,45
The Journal of Finance,44


time: 78 ms (started: 2025-05-18 21:52:14 +01:00)


### Distinct authors

In [293]:
b = sc.Bib2df(p)
len(b.df)

6118

time: 391 ms (started: 2025-05-18 21:41:26 +01:00)


In [353]:
authors = b.distinct('author')
# authors = b.distinct('title')
authors[:10]

['',
 '...',
 '2017, Undefined',
 "A, Hidalgo C{\\'{e}}sar",
 'A, Yildirim Muhammed',
 'ACIA, Dublin',
 'AIPSO',
 'AON',
 'ASPE',
 'ATC']

time: 16 ms (started: 2025-05-18 22:13:53 +01:00)


In [354]:
import pandas as pd
import latexcodec
rbrace = re.compile(r'{{(.)}}')
rinitials = re.compile(r' ([A-Z])(\s|$)')
def latex_to_unicode(s):
    try:
        n = rbrace.sub(r'\1', s.encode('latin1').decode('latex'))
        return rinitials.sub(r' \1.\2', n)
    except Exception:
        return s  # fallback for bad input

time: 0 ns (started: 2025-05-18 22:13:55 +01:00)


In [355]:
rinitials.sub(r' \1.\2', 'Person JR')

'Person JR'

time: 15 ms (started: 2025-05-18 22:13:57 +01:00)


In [358]:
name_mapper = pd.DataFrame({'original': authors, 'revised': map(latex_to_unicode, authors)})

time: 1.59 s (started: 2025-05-18 22:14:30 +01:00)


In [362]:
name_mapper.to_csv('dm.csv')

time: 31 ms (started: 2025-05-18 22:15:46 +01:00)


In [361]:
un = list(name_mapper.revised.value_counts().keys())
un[:19]

['Carson, James M.',
 'Boonen, Tim J.',
 'Babbel, David F.',
 'Kunreuther, Howard C.',
 'Cummins, J. David',
 'Lee, J.',
 'Barndorff-Nielsen, Ole E.',
 'Hardy, Mary R.',
 'Lange, Jeffrey T.',
 'Lin, X. Sheldon',
 'Goodman, Steven N.',
 'Jewell, William S.',
 'Klugman, Stuart A.',
 'Gerber, Hans U.',
 'Johnson, Norman L.',
 'Born, Patricia H.',
 'Epstein, Larry G.',
 'Aliprantis, Charalambos D.',
 'Burrus, C. Sidney']

time: 0 ns (started: 2025-05-18 22:15:18 +01:00)


In [352]:
fGT(name_mapper.sample(50).sort_values('original'))

index,original,revised
21,{10-K},{10-K}
23,{1986-94 Credit Risk Loss Experience Study: Private Placement Bonds},{1986-94 Credit Risk Loss Experience Study: Private Placement Bonds}
63,{A Brief Memoire of My Involvement with Aon Benfield and its Predecessors},{A Brief Memoire of My Involvement with Aon Benfield and its Predecessors}
65,{A Central Limit Theorem for Convex Sets},{A Central Limit Theorem for Convex Sets}
511,{All adapted topologies are equal},{All adapted topologies are equal}
518,{Allocation of Firm-Energy Rights among Hydro agents using Cooperative Game Theory : an Aumann-Shapley approach},{Allocation of Firm-Energy Rights among Hydro agents using Cooperative Game Theory : an Aumann-Shapley approach}
615,{An assessment of the long-term economic impact of stronger capital and liquidity requirements},{An assessment of the long-term economic impact of stronger capital and liquidity requirements}
898,"{Big data: The next frontier for innovation, competition, and productivity}","{Big data: The next frontier for innovation, competition, and productivity}"
926,{Bond and CDS Pricing with Recovery Risk I: The Stochastic Recovery Merton Model},{Bond and CDS Pricing with Recovery Risk I: The Stochastic Recovery Merton Model}
959,{Building Watson: An Overview of the DeepQA Project},{Building Watson: An Overview of the DeepQA Project}


time: 63 ms (started: 2025-05-18 22:11:52 +01:00)


In [298]:
list(map(normalize_author, authors[10:30]))

['AXCO',
 'Aaker, D.',
 'Aalen, O. O.',
 'Aas, K.',
 'Aas, K.',
 'Aase, K. K.',
 'Abanin, D.',
 'Abascal, F.',
 'Abate, J.',
 'Abatzoglou, A.',
 'Abbott, B. P.',
 'Abdel-Hameed, M.',
 'Abdikerimova, S.',
 'Abowd, J. M.',
 'Abraham, A.',
 'Abraham, J.',
 'Abraham, P.',
 'Abraham, R.',
 'Abraham, S.',
 'Abramowitz, M.']

time: 0 ns (started: 2025-05-18 21:44:19 +01:00)


In [296]:
import re
import latexcodec

def normalize_author(name: str) -> str:
    """Replace first names with initials and use periods."""
    try:
        name = name.encode('latin1').decode('latex')
    except Exception:
        pass  # leave as-is if decoding fails

    name = name.strip()
    if ',' not in name:
        return name  # likely an organization

    last, first = map(str.strip, name.split(',', 1))
    parts = re.split(r'[\s\-]+', first)
    initials = [p[0].upper() + '.' for p in parts if p and p[0].isalpha()]
    return f"{last}, {' '.join(initials)}"


time: 0 ns (started: 2025-05-18 21:43:28 +01:00)


In [300]:
import re
import latexcodec
from collections import defaultdict

def latex_to_unicode(s: str) -> str:
    try:
        return s.encode('latin1').decode('latex')
    except Exception:
        return s

def extract_parts(name: str):
    """Returns (last, [first parts]), or None if not a person."""
    if ',' not in name:
        return None
    last, first = map(str.strip, name.split(',', 1))
    parts = re.split(r'[\s\-]+', first)
    return last, parts

def normalize_name(name: str, upgrades: dict[str, list[str]] = None) -> str:
    name = latex_to_unicode(name).strip()
    parts = extract_parts(name)
    if parts is None:
        return name  # organization
    last, first_parts = parts

    if upgrades and last in upgrades:
        # try to upgrade initials to full names
        full_parts = upgrades[last]
        new_parts = []
        for i, p in enumerate(first_parts):
            if len(p) == 2 and p[1] == '.' and i < len(full_parts):
                new_parts.append(full_parts[i])
            else:
                new_parts.append(p)
        first = ' '.join(new_parts)
    else:
        first = ' '.join(first_parts)

    return f"{last}, {first}"

def build_upgrades(names: list[str]) -> dict[str, list[str]]:
    d = defaultdict(set)
    for name in names:
        parts = extract_parts(latex_to_unicode(name))
        if parts:
            last, first_parts = parts
            for p in first_parts:
                if len(p) > 2 or p[-1] != '.':
                    d[last].add(tuple(first_parts))
                    break
    # choose the longest variant for each last name
    return {k: max(v, key=len) for k, v in d.items()}


time: 16 ms (started: 2025-05-18 21:45:21 +01:00)


In [302]:
 authors[10:30]

['AXCO',
 'Aaker, David',
 'Aalen, Odd O.',
 'Aas, K',
 'Aas, Kjersti',
 'Aase, Knut K.',
 'Abanin, D.',
 'Abascal, Federico',
 'Abate, Joseph',
 'Abatzoglou, Alexander',
 'Abbott, B P',
 'Abdel-Hameed, Mohamed',
 'Abdikerimova, Samal',
 'Abowd, John M.',
 'Abraham, Andreas',
 'Abraham, Jon',
 'Abraham, Paul',
 'Abraham, R',
 'Abraham, Sarah',
 'Abramowitz, Milton']

time: 16 ms (started: 2025-05-18 21:45:45 +01:00)


In [301]:
upgrades = build_upgrades( authors[10:30])
upgrades

{'Aaker': ('David',),
 'Aalen': ('Odd', 'O.'),
 'Aas': ('Kjersti',),
 'Aase': ('Knut', 'K.'),
 'Abascal': ('Federico',),
 'Abate': ('Joseph',),
 'Abatzoglou': ('Alexander',),
 'Abbott': ('B', 'P'),
 'Abdel-Hameed': ('Mohamed',),
 'Abdikerimova': ('Samal',),
 'Abowd': ('John', 'M.'),
 'Abraham': ('Andreas',),
 'Abramowitz': ('Milton',)}

time: 16 ms (started: 2025-05-18 21:45:22 +01:00)


## Files

In [303]:
ffdf = b.file_field_df

time: 562 ms (started: 2025-05-18 21:47:52 +01:00)


In [304]:
len(ffdf)

6079

time: 0 ns (started: 2025-05-18 21:47:59 +01:00)


In [307]:
ffdf.head()

,i,drive,file,type
i,,,,
1,1,C\,/S/Library/Schmidt/2003_The true invariant of ...,pdf
2,2,C\,/S/Library/Bahnemann/2015_Distributions for Ac...,pdf
2,2,C\,/S/Library/Bahnemann/2015_Distributions for Ac...,pdf
3,3,C\,/S/Library/Khoshnevisan/2011_Topics in Probabi...,pdf
4,4,C\,/S/Library/Mahul/2003_Efficient Risk Sharing w...,pdf


time: 0 ns (started: 2025-05-18 21:48:25 +01:00)


In [314]:
ffdf.head().drop(columns='i').file.values

array(['/S/Library/Schmidt/2003_The true invariant of an arbitrage free portfolio.pdf',
       '/S/Library/Bahnemann/2015_Distributions for Actuaries.pdf',
       '/S/Library/Bahnemann/2015_Distributions for Actuaries(2).pdf',
       '/S/Library/Khoshnevisan/2011_Topics in Probability Levy Processes.pdf',
       '/S/Library/Mahul/2003_Efficient Risk Sharing within a Catastrophe Insurance Pool.pdf'],
      dtype=object)

time: 0 ns (started: 2025-05-18 21:50:10 +01:00)


## Bibtex parser read

In [42]:
scbt.LIBRARY_BIB, p, p.exists(), scbt.BOOKS_BIB

('library.bib', WindowsPath('library.bib'), True, 'books.bib')

time: 0 ns (started: 2025-05-17 10:12:17 +01:00)


In [43]:
bdb = scbt.bibtex_read() # too slow... scbt.BOOKS_BIB)

time: 437 ms (started: 2025-05-17 10:12:29 +01:00)


In [44]:
len(bdb)

TypeError: object of type 'BibDatabase' has no len()

time: 47 ms (started: 2025-05-17 10:12:36 +01:00)
